In [2]:
import os
import re
import sqlite3
import json
import zipfile
from bs4 import BeautifulSoup
from lxml import etree


In [3]:
conn = sqlite3.connect('example.db')
cursor = conn.cursor()

# Create table
cursor.execute('''CREATE TABLE IF NOT EXISTS powerpoint
                (id INTEGER PRIMARY KEY, drill LIST, years TEXT, age TEXT, repayment TEXT, stipend TEXT, estimated TEXT, program TEXT)''')

In [15]:
# Function to extract text from PowerPoint files
def extract_text_from_pptx(file_path):
    text = ""
    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        for name in zip_ref.namelist():
            if name.startswith('ppt/slides/slide'):
                with zip_ref.open(name) as slide_file:
                    slide_content = slide_file.read()
                    # Extract text from XML content
                    xml_content = etree.fromstring(slide_content)
                    text += " ".join(xml_content.xpath(".//*[local-name()='t']//text()"))

    return text

# Function to find numbers and their surrounding words
def find_numbers_with_context(text):
    pattern = r'(\b\w+\b\s+){0,5}\b(\d+)\b(\s+\b\w+\b){0,5}'
    matches = re.findall(pattern, text)
    results = []
    for match in matches:
        # Combine the surrounding words with the number
        context = " ".join(match)
        results.append(context)
    return results

In [16]:
# Iterate through PowerPoint files in the directory
pptx_directory = "/Users/annivoigt/Documents/coding/python/DS-S_Ship_Testing/Powerpoints & DB/powerpoints"

In [18]:
# Function to convert list of numbers to dictionary format
def dict_list(list_num):
    my_information = {"drill":[],"years":[], "age":[], "repayment":[], "stipend":[], "estimated":[],'program':[]}
    for i in list_num:
        if 'drill' in i:
            num = re.findall(r'\d+', i)
            my_information["drill"].append(num[0])
        if 'years' in i:
            num = re.findall(r'\d+', i)
            my_information["years"].append(num[0])
        if 'age' in i:
            num = re.findall(r'\d+', i)
            my_information["age"].append(num[0])
        if 'repayment' in i:
            num = re.findall(r'\d+', i)
            my_information["repayment"].append(num[0])
        if 'stipend' in i:
            num = re.findall(r'\d+', i)
            my_information["stipend"].append(num[0])
        if 'estimated' in i:
            num = re.findall(r'\d+', i)
            my_information["estimated"].append(num[0])
        if 'program' in i:
            num = re.findall(r'\d+', i)
            my_information["program"].append(num[0])
    return my_information

In [20]:
# Function to insert data into SQLite database
def sql_add(my_information):
    age_json = json.dumps(my_information['age'])
    drill_json = json.dumps(my_information['drill'])
    years_json = json.dumps(my_information['years'])
    repayment_json  = json.dumps(my_information['repayment'])
    stipend_json = json.dumps(my_information['stipend'])
    estimate_json = json.dumps(my_information['estimated'])
    program_json = json.dumps(my_information['program'])

    cursor.execute('INSERT INTO powerpoint (drill, years, age, repayment, stipend, estimated, program) VALUES (?, ?, ?, ?, ?, ?, ?)',\
            (drill_json, years_json,\
            age_json, repayment_json, \
            stipend_json, estimate_json, program_json))

In [21]:
list_num = []
for filename in os.listdir(pptx_directory):
    if filename.endswith(".pptx"):
        file_path = os.path.join(pptx_directory, filename)
        text = extract_text_from_pptx(file_path)
        numbers_with_context = find_numbers_with_context(text)
        if numbers_with_context:
            for context in numbers_with_context:
                list_num.append(context)
                info = dict_list(list_num)
                sql_add(info)
        else:
            numbers = re.findall(r'\b\d+\b', text)
            if numbers:
                for number in numbers:
                    list_num.append(number)

In [22]:
# Commit the transaction
conn.commit()

# Close connection
conn.close()

In [24]:
#This is how users could connect to the database and look at what exists in the database 
# -- we can create analytic products based off of this information
conn = sqlite3.connect('example.db')
cursor = conn.cursor()



def query_database(conn, query):
    cursor = conn.cursor()
    cursor.execute(query)
    rows = cursor.fetchall()
    return rows


# Example query to select all data from the 'person' table
query = 'SELECT drill FROM powerpoint'

# Query the database
results = query_database(conn, query)
print(results)

# Close connection
conn.close()

[('["000"]',), ('[]',), ('[]',), ('[]',), ('[]',), ('[]',), ('[]',), ('[]',), ('[]',), ('[]',), ('[]',), ('[]',), ('[]',), ('[]',), ('[]',), ('[]',), ('[]',), ('[]',), ('["000"]',), ('["000"]',), ('["000"]',), ('["000"]',), ('["000"]',), ('["000"]',), ('["000"]',), ('["000"]',), ('["000"]',), ('["000"]',), ('["000"]',), ('["000"]',), ('["000"]',), ('["000"]',), ('["000"]',), ('["000"]',), ('["000"]',), ('["000"]',), ('["000"]',), ('["000"]',), ('["000"]',), ('["000"]',), ('["000"]',), ('["000"]',), ('["000"]',), ('["000"]',), ('["000"]',), ('["000"]',), ('["000"]',), ('["000"]',), ('["000"]',), ('["000"]',)]
